### 智能体的工作流程

LangChain的智能体(Agent)工作流程具体如下，以继承自`ActionAgent`类型为例：
- 获取用户输入
- 调用对应的AgentTools
- 检索并获取输出

LangChain中的智能体分为如下类型：
- `ActionAgent`：用于执行特定的动作，如打开应用、发送邮件、检索信息等
- `Plan and Execute Agent`：用于生成规划与执行
- `Autonomous Agent`：用于执行自动化任务
- `Generative Agent`：用于执行生成式任务

### 智能体类型中的中点：Plan and Execute Agent
它把Agent分类为两个部分：规划器和执行器。规划器具有一个语言模型，用做推理和提前计划许多步骤。执行器会分析输入，根据初始化时选定的工具让智能体以最合适的方式完成工作。

### Agent运行机制
`Agent`经过初始化后，产生由AI计划和决策后，再由`AgentExecutor`执行来自于智能体所规划的具体行为。
- 继承来自`Chain`类型，因此`AgentExecutor`可以继承并访问来自`Chain`的方法和属性的同时，也可以使用来自其自身的方法和属性。
- 成员变量`agent`：将传入的智能体赋值给`agent`，以便在`AgentExecutor`中使用。该智能体可以是`BaseSingleActionAgent`或`BaseMultiActionAgent`。此智能体负责产生计划和动作以供执行。
- agent的调用：`take_next_step`是`AgentExecutor`的核心方法，负责`AgentExecutor`的每一步索要具体执行的内容。`take_next_step`会被`agent`的`plan`方法所调用。
- 工具和验证管理：`AgentExecutor`负责提供给agent的工具是否兼容，以及管理agent的执行，包括max_iter和最大执行时间等。一个agent多做对应一个工具，这些工具在内部方法`name_to_tool_map`中的字典进行查找和执行。

In [1]:
### Agent组件入门示例
import os
# 导入大语言模型
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType
)
from langchain.llms import OpenAI

llm = OpenAI()

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [2]:
# 加载必要工具？
tools = load_tools(["serpapi","llm-math"],llm=llm)

In [3]:
# 初始化智能体
agent = initialize_agent(tools,llm,AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [4]:
# 询问智能体
agent.run(
    "What is Apple R1 Chip made for?"
)

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 The Apple R1 Chip is a recent technology, so it would be best to use a search engine to find information about it.
Action: Search
Action Input: "Apple R1 Chip"
Observation: ["The R1 is an SPU (Sensor Processing Unit) which works in concert with the M2's integrated GPU (the R1 being the innie, and the M2 being the ...", 'The Apple R1, not the main M2 chip, processes a continuous stream of real-time data feed to the Vision Pro by its twelve cameras, five sensors, ...', 'And the brand-new R1 chip is specifically dedicated to process input from the cameras, sensors, and microphones, streaming images to the displays within 12 ...', "To support R1's high-speed processing, SK hynix has developed the custom 1-gigabit DRAM. The new DRAM is known to have increased the number of ...", 'R1 streams new images to the displays within 12 milliseconds — 8x faster than the blink of an eye. Apple Vision Pro is designed for all-day use ...', "A close-up and labelin

'The Apple R1 Chip is primarily used for processing real-time data from cameras, sensors, and microphones for head and hand tracking and 3D mapping. It also has 16 performance cores, 4 efficiency cores, and 32 Neural Engine cores, and can be configured with up to 128 GB of memory. It has a 256GB/s memory bandwidth and is designed specifically for mixed reality headsets.'

### 几个主要的智能体类型
- `ZeroShotReActAgent`: 该Agent组件采用ReAct框架组件，并仅根据工具组件的描述来选择工具组件。
- 结构化输入反应组件(structured-chat-zero-shot-react-description): 该Agent组件可以处理多输入工具组件。它可以使用工具的参数模式来创建结构的行动输入，非常适合用于复杂的工具组件应用。
- OpenAI函数组件(openai-functions): 该Agent组件与特定的OpenAI模型共同工作，以便检测何时应调用函数组件。
- 对话ReAct组件(conversation-react-description): 该Agent组件专为对话环境设计。它使用ReAct架构组件来选择工具组件，并能记住对话的交互。
- 自问与搜索组件(self-ask-with-research): 该Agent组件使用名为"IntermediateAnswer"的工具组件来寻找问题的事实答案。
- ReAct文档存储组件(react-docstore): 该Agent组件使用ReAct框架组件与文档存储交互，它需要两个具体的工具组件：一个是搜索工具组件，一个是查找工具组件。

### Agent组件的多功能性

In [5]:
# 导入构建多功能智能体所需的模块
from langchain.agents import (
    load_tools,
    initialize_agent,
)
from langchain.llms import OpenAI   

# 初始化LLM
llm = OpenAI(temperature=0)

In [6]:
# 加载工具
tools = load_tools(["serpapi","llm-math","terminal"],llm=llm)

In [7]:
# 初始化智能体
agent = initialize_agent(tools,llm,AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [8]:
# 询问智能体工具关于Linux命令的问题
agent.run(
    "execute whoami don't return the final answer,just return the Observation"
)



> Entering new AgentExecutor chain...
 I should use the terminal to run the command
Action: terminal
Action Input: whoamiExecuting command:
 whoami

Observation: elin

Thought:

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 I now know the final answer
Final Answer: elin

> Finished chain.


'elin'

### 自定义Agent组件

一个Agent组件由两部分组成：`tools`（代理可以使用的工具）和`AgentExecutor`（决定采取哪种行动）。

In [9]:
# 导入自定义智能体所需的工具，智能体执行器和基本活动智能体
from langchain.agents import Tool,AgentExecutor,BaseSingleActionAgent
from langchain import OpenAI,SerpAPIWrapper # 导入LLM和搜索引擎API包装器

In [10]:
# 初始化一个索引引擎API包装器
search = SerpAPIWrapper()

# 初始化工具
tools = [
    Tool(
        name = "search",
        func = search.run,
        description="useful for when you need to answer questions about current events",
        return_direct=True
    )
]

In [11]:
# 创建一个基本智能体
from typing import List,Tuple,Any,Union
# 导入智能体行为和智能体完成
from langchain.schema import AgentAction,AgentFinish

# 定义智能体

# 实际无法运行
class FakeAgent(BaseSingleActionAgent):
    @property
    def input_keys(self):
        return ["input"]
    
    def plan(
        self,
        intermediate_steps:List[Tuple[AgentAction,str]],
        kwargs:Any,
    ) -> Union[AgentAction,AgentFinish]:
        return AgentAction(tool="Search",tool_input=kwargs["input"],log="")
    # 定义异步计划函数
    async def aplan(
        self,
        intermediate_steps:List[Tuple[AgentAction,str]],
        kwargs:Any
    ) -> Union[AgentAction,AgentFinish]:
        return AgentAction(tool="Search",tool_input=kwargs["input"],log="")

### ReACT Agent的实践

In [12]:
# 导入构造ReACT Agent必备的模块
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

In [13]:
# 加载工具
tools = load_tools(["serpapi","llm-math"],llm=llm)

In [14]:
# 初始化智能体
agent = initialize_agent(
    tools,
    llm,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [15]:
agent.run(
    "When did Apple release iOS7 ?"
)



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: Apple iOS7 release date
Observation: September 18, 2013
Thought: I now know the final answer.
Final Answer: September 18, 2013

> Finished chain.


'September 18, 2013'

In [17]:
# 使用ChatLLM
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)

agent = initialize_agent(
    tools,
    chat_model,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run(
    "Who is the designer of iPhone5s ?"
)



> Entering new AgentExecutor chain...
I need to search for the designer of iPhone5s.
Action: Search
Action Input: "designer of iPhone5s"
Observation: The iPhone 5s is a smartphone that was designed and marketed by Apple Inc.
Thought:I need to search for the specific designer of the iPhone 5s.
Action: Search
Action Input: "iPhone 5s designer"
Observation: The iPhone 5s originally shipped with iOS 7, which introduced a revamped visual appearance among other new features. Designed by Jony Ive, iOS 7 departed from skeuomorphic elements used in previous versions of iOS in favor of a flat, colorful design.
Thought:I now know the final answer
Final Answer: Jony Ive is the designer of iPhone 5s.

> Finished chain.


'Jony Ive is the designer of iPhone 5s.'

### Agent组件的功能增强
- 记忆功能增强：为OpenAI Functions Agent组件引入记忆功能是一次重大的突破。这不仅让Agent组件组件记住先前的对话内容，还使其在执行连续任务时表现更为出色。例如，Agent组件能从工具中提取的3个质数，并进行相乘。在此过程中，Agent组件还有能力验证中间结果，如确认其输出是否为质数。
- 与向量存储库的融合：为了让Agent组件更有效地与向量存储库互动，建议引入一个`RetrievalQA`，并将其纳入整体Agent的工具集中。此外，Agent组件还能与多个向量数据库进行交互，并在它们之间实现路由。这使Agent组件在数据访问和处理方面具备更广的能力。

### Agent组件的记忆功能增强

In [1]:
# 加载基础工具链
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper,
    SQLDatabase,
)
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent,Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

In [12]:
# 处理化llm和工具链
llm = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")
# 初始化serp搜索包装器
search = SerpAPIWrapper()
# 初始化llm数学链
llm_math_chain = LLMMathChain.from_llm(llm=llm,verbose=True)

# 初始化DB
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db"
)
# 初始化DB链
db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)
# 定义工具链
tools = [
    Tool(
        name = "Search",
        func = search.run,
        description = "useful for when you need to answer questions about current events",
    ),
    Tool(
        name = "Calculator",
        func = llm_math_chain.run,
        description = "useful for when you need to answer questions about math",
    ),
    Tool(
        name = "FoorBar-DB",
        func = db_chain.run,
        description = "useful for when you need to answer questions about FoorBar,Input Should be in the form of a question containing full context",
    )
]

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
# 加载记忆工具和智能体参数
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# 初始化智能体参数
agent_kwargs = {
    "extra_prompt_messages":[
        MessagesPlaceholder(variable_name="memory")
    ]
}
# 初始化记忆
memory = ConversationBufferMemory(
    memory_key = "memory",
    return_messages = True
)

In [23]:
# 初始化智能体
agent = initialize_agent(
    tools,
    llm,
    AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory = memory
)

In [24]:
# 运行第一次智能体对话
agent.run("Hello")



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


'Hello! How can I assist you today?'

In [25]:
# 运行第二次智能体对话
agent.run("My name is Elin")



> Entering new AgentExecutor chain...
Nice to meet you, Elin! How can I help you today?

> Finished chain.


'Nice to meet you, Elin! How can I help you today?'

In [26]:
# 运行第三次智能体对话
agent.run("Whats my name?")



> Entering new AgentExecutor chain...
Your name is Elin.

> Finished chain.


'Your name is Elin.'

### Agent组件的检索能力增强

In [27]:
# 加载基础工具链和嵌入等
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [28]:
# 初始化LLM
llm = OpenAI(temperature=0)

In [31]:
# 创建向量存储库


from langchain.document_loaders import TextLoader
# 加载文档
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()

# 创建分割器
text_spliter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)

# 分割文档
texts = text_spliter.split_documents(documents)

# 初始化嵌入
embeddings = OpenAIEmbeddings()

# 创建向量搜索引擎

doc_search = Chroma.from_documents(
    texts,
    embeddings,
    collection_name="state-of-union"
)

In [33]:
# 实例化检索QA链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type = "stuff",
    retriever = doc_search.as_retriever()
)

In [34]:
# 加载WebBaseLoader
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://pybind11.readthedocs.io/en/stable/basics.html")

docs = loader.load()

# 分割文档
texts = text_spliter.split_documents(docs)

# 创建向量搜索引擎
doc_search = Chroma.from_documents(
    texts,
    embeddings,
    collection_name="pybind11-docs"
)

# 实例化检索QA链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type = "stuff",
    retriever = doc_search.as_retriever()
)

In [35]:
# 正式开始创建智能体
from langchain.agents import initialize_agent,Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain,SerpAPIWrapper

In [36]:
# 定义可用工具
tools = [
    Tool(
        name = "Search of Union QA System",
        func = search.run,
        description = "useful for when you need to answer questions"
          "about the most recent state of the union address.Input should be a fully"
          "formed question.",
    ),
    Tool(
        name = "PyBind QA System",
        func = qa_chain.run,
        description = "useful for when you need to answer questions about pybind11 docs."
          "Input should be a fully formed question.",
    )
]

In [37]:
# 初始化智能体
agent = initialize_agent(
    tools,
    llm,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [39]:
# 开始提问
agent.run("Where is the ShenYang University located?")



> Entering new AgentExecutor chain...
 I should use the Search of Union QA System to find the answer.
Action: Search of Union QA System
Action Input: "Where is ShenYang University located?"
Observation: ['Shenyang University is a university in Shenyang, Liaoning, China under the provincial government. It has a student body of more than 30,000 students and has about 1260 full-time teachers.', 'Shenyang University type: University in Shenyang, China.', 'Shenyang University entity_type: education, locations, local_nav.', 'Shenyang University kgmid: /m/03px1v.', 'Shenyang University address: 21 Wang Hua Nan Jie, Da Dong Qu, Shen Yang Shi, Liao Ning Sheng, China, 110096.', 'Shenyang University notable_alumni: Märt Läänemets, Mariam Namata, Xiaohai Wang.', 'Shenyang University province: Liaoning.', 'Shenyang University founded: 1906.', 'Shenyang University phone: +86 24 6226 7112.', 'Shenyang University http: //www.syu.edu.cn/.', "The Main Campus located in Shenyang City's Dadong District.

'Shenyang University is located in Shenyang, Liaoning, China.'

In [40]:
# 开始提问PyBind11相关问题
agent.run("How can i make a hello world function using PyBind11?")



> Entering new AgentExecutor chain...
 I should use the PyBind QA System to find the answer.
Action: PyBind QA System
Action Input: "How can I make a hello world function using PyBind11?"
Observation: 

To make a hello world function using PyBind11, you can follow these steps:

1. Create a new file named "hello.cpp" with the following contents:

#include <pybind11/pybind11.h>

std::string hello() {
    return "Hello, world!";
}

PYBIND11_MODULE(hello, m) {
    m.doc() = "pybind11 hello plugin"; // optional module docstring

    m.def("hello", &hello, "A function that returns a hello message");
}

2. Compile the code using the following command on Linux:

$ c++ -O3 -Wall -shared -std=c++11 -fPIC $(python3 -m pybind11 --includes) hello.cpp -o hello$(python3-config --extension-suffix)

3. Import the module in Python and call the hello function:

import hello
print(hello.hello())

This should print out "Hello, world!" in the console.
Thought: I now know the final answer.
Final Answer: To

'To make a hello world function using PyBind11, you can follow the steps outlined above.'